In [1]:
import sys
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## Get Data from mySQL database
### synapse - synapsecombined
contains the pre and post synapses, the type, and the synapse contin number
### obj - object
contains the x, y, and image_num of each section in the contin

In [2]:
def read_in_data( field, ind_lst ):

    skip = True
    out = []
    with open( 'thumb/Emmons_Data/N2U.sql' ) as f:
        for line in f:
            if line.startswith('INSERT INTO'):
                
                if field in line.split()[2]:
                    skip = False
                else:
                    skip = True
            elif skip == False:
                if line == '\n':
                    skip = True
                else:
                    ext = []
                    for i in ind_lst:
                        ext.append( line.rstrip('\r\n').split(', ')[i].lstrip(' \'')\
                            .rstrip('\'').rstrip(',').rstrip(';').rstrip(')')\
                            .lstrip('(') )
                    out.append( ext )
    return out

In [3]:
synapses = pd.DataFrame( read_in_data( 'synapsecombined', [1,2,3,19] ), columns=['pre', 'post', 'type', 'contin'] )
print synapses

         pre            post        type contin
0       ASJL       PVQL,ASKL    chemical   5641
1       AIAL  AIML,ASJL,ASKL    chemical   5642
2        AVM            PVCL    chemical   9582
3        AVM  PVCL,AVBL,PVNL    chemical   9583
4       ASHR       ADAR,AIAR    chemical    369
5       RIBR            AIZR    chemical    370
6       AIZL      RIAL,SMBDL    chemical    371
7       AIZL       AIBR,AVER    chemical    372
8       HSNR       RIML,ASHL    chemical    373
9       AWAL       AFDL,ASEL    chemical    374
10       AVM       PVCL,AVBL    chemical    375
11      PVNL       AVJL,BDUR    chemical    377
12      RIML       AIBR,AVER    chemical    379
13      ASEL            AIYL    chemical    380
14      AIYL       RIAL,RIBL    chemical    381
15      ADFL       RIAL,AWBL    chemical    382
16      SDQL             RIS    chemical    383
17      SDQR       AVAL,AIBL    chemical    384
18      AIYR  RIAR,AIZR,RIBR    chemical    385
19      AWAR  AIYR,AIZR,ADFR    chemical

In [4]:
obj = pd.DataFrame( read_in_data( 'object', [1,2,4,5] ), columns=['x', 'y', 'img_num', 'contin'] )
print obj

           x     y   img_num contin
0        776  2080  N2UNR176      1
1        785  2035  N2UNR177      1
2        798  1980  N2UNR178      1
3        826  1934  N2UNR179      1
4        844  1912  N2UNR180      1
5        885  1882  N2UNR181      1
6        901  1851  N2UNR182      1
7        915  1799  N2UNR183      1
8        765  2121  N2UNR175      1
9        750  2178  N2UNR174      1
10       767  2228  N2UNR173      1
11      3528  5540  N2UNR183    222
12      3542  5530  N2UNR182    222
13      3542  5530  N2UNR181    222
14      3571  5529  N2UNR180    222
15      3597  5515  N2UNR179    222
16      3633  5511  N2UNR178    222
17      3668  5501  N2UNR177    222
18      3696  5498  N2UNR176    222
19      3714  5475  N2UNR175    222
20      3739  5457  N2UNR174    222
21      3752  5433  N2UNR173    222
22      3283  5506  N2UNR183      3
23      3289  5497  N2UNR182      3
24      3297  5489  N2UNR181      3
25      3332  5477  N2UNR180      3
26      3370  5470  N2UNR179

In [5]:
## Probably not going to use this
contin = pd.DataFrame( read_in_data( 'contin', [1, 8, 9] ), columns=['contin','section1','section2'] )
print contin

     contin section1 section2
0       294      164      171
1       315      150      153
2       295      164      172
3       314      150      154
4        99      173      177
5       134      173      179
6       292      164      166
7       316      150      154
8       302      164      169
9       319      140      145
10     2206      131      133
11     4606      121      127
12       93        1      185
13      332      140      142
14     8538        0        0
15     5725      411      798
16     5771      280      574
17      141      128      183
18      113       61       96
19     5713        1      326
20      159        1      283
21       42      172      183
22      230       51      183
23     5733        1      329
24       43        1      287
25      270       70      183
26       91        1      183
27      219        2      183
28      178        1      183
29      272        2      183
...     ...      ...      ...
9858   9859        0        0
9859   986

## Section Numbering
I need to clarify how this is actually going to play out, but for now I'm using the img_num to define the Z coordinate of a synapse. It seems like there are three series types: NR = nerve ring, NC = neural cord, DC = dorsal commisure. From the wormwiring diagrams online it appears that NR comes before NC which comes before DC but the img number gets reset to 0 with each series change. I'm going to add 200 to the NC img numbers. Not sure what to do with DC yet so I'll leave them alone for now.

In [6]:
full_df = pd.merge( syn, obj, on='contin' ).sort_values( 'contin' )
full_df = full_df[ full_df['contin'] != '0' ]
## split img_num into series name and 'section'
series, section = [], []
for img in full_df[['img_num']].values:
    series_name, section_num = img[0][3:5], int( img[0][5:] )
    ## Define the section number each series starts at here
    if series_name == 'VC':
        section.append( section_num + 200 ), series.append( series_name )
    elif series_name == 'NR':
        section.append( section_num ), series.append( series_name )
    else:
        section.append( section_num ), series.append( series_name )
full_df['series'], full_df['section'] = series, section
print full_df

NameError: name 'syn' is not defined

In [ ]:
## convert img_num to section. Relationship stored in 'image' table
print read_in_data( 'image', [0,1])

In [ ]:
## Average coordinates between a contin
averaged_con = []
last, x_avg, y_avg, sec_avg = 0, None, None, None
for row in full_df.values:
    if int(row[3]) != last:
        averaged_con.append( [row[0], row[1], row[2], row[7], last, x_avg, y_avg, sec_avg] )
        last, x_avg, y_avg, sec_avg = int(row[3]), int(row[4]), int(row[5]), int(row[8])
    else:
        x_avg = np.mean( [x_avg, int(row[4])] )
        y_avg = np.mean( [y_avg, int(row[5])] )
        sec_avg = np.mean( [sec_avg, int(row[8])] )
averaged_df = pd.DataFrame( averaged_con[1:], columns=['pre','post','type','series','contin','x','y','z'] )
print averaged_df

## Graphing

In [ ]:
neuron = 'AVAR'
averaged_df = averaged_df[ averaged_df['post'].str.contains( neuron ) ].append( \
            averaged_df[ averaged_df['pre'].str.contains( neuron ) ] )\
            [['pre', 'post', 'x', 'y', 'type', 'z', 'contin']]
print np.shape( averaged_df )
pre, post, x, y, z, typ = averaged_df['pre'].values, averaged_df['post'].values, averaged_df['x'].values, \
                     averaged_df['y'].values, averaged_df['z'].values, averaged_df['type'].values
plt.figure(figsize=(20,5)) 
for i in range( len( pre ) ):
    if typ[i] == 'electrical':
        plt.plot(z[i], x[i], 'co')
        if pre[i] == neuron:
            plt.text(z[i], float(x[i]), post[i], fontsize=10, alpha=0.6)
        else:
            plt.text(z[i], float(x[i]), pre[i], fontsize=10, alpha=0.6)
    elif pre[i] == neuron:
        plt.plot(z[i], x[i], 'ro')
        plt.text(z[i], float(x[i]), post[i], fontsize=10, alpha=0.6)
    else:
        plt.plot(z[i], x[i], 'mo')
        plt.text(z[i], float(x[i]), pre[i], fontsize=10, alpha=0.6)
plt.title( neuron ), plt.xlabel( 'section #' ), plt.ylabel( 'X');

